In [1]:
#pragma cling add_include_path("./thirdparty/kdb/include")
#pragma cling add_include_path("./thirdparty/kdb/include/internal")
#pragma cling load("./thirdparty/kdb/build/libkdb.so") 

#include </home/thorad/Core/Projects/Qenv/notebooks/thirdparty/kdb/include/kdb_cpp.h>
#include <string>
#include <iostream>

In [2]:
kdb::Connector kcon;
#define HOST_ADDR "127.0.0.1"
#define HOST_PORT 5050

In [3]:
if (!kcon.connect(HOST_ADDR, HOST_PORT))
        return -1;

In [4]:
kdb::Result rs = kcon.sync(".server.DumStep[((0;1);(1;5);(2;4);(3;5))]");

In [5]:
kdb::Type st = rs.type();

In [8]:
st colvla = rs.get();

input_line_15:2:4: error: expected ';' after expression
 st colvla = rs.get();
   ^
   ;
input_line_15:2:5: error: use of undeclared identifier 'colvla'
 st colvla = rs.get();
    ^
input_line_15:2:17: error: no matching member function for call to 'get'
 st colvla = rs.get();
             ~~~^~~
./thirdparty/kdb/include/internal/kdb_result.h:60:11: note: candidate template ignored: couldn't infer template argument 'T'
        T get() const {
          ^
./thirdparty/kdb/include/internal/kdb_result.h:71:11: note: candidate template ignored: couldn't infer template argument 'T'
        U get() const {
          ^


Interpreter Error: 

In [9]:
kdb::Result res = kcon.sync("select from tbl_test");

In [10]:
kdb::Table tbl = res.get_table();

In [11]:
// Traverse a column
kdb::Vector<kdb::Type::Long> column = tbl.get_column<kdb::Type::Long>(0);
std::cout << "=======================================\nColumn 0: ";
for (auto const &it : column) {
    std::cout << it << ' ';
}
std::cout << '\n';

Column 0: 1 2 3 4 


In [5]:
#pragma cling add_include_path("./thirdparty/libtorch/include")
#pragma cling add_include_path("./thirdparty/libtorch/include/torch/csrc/api/include")
#pragma cling add_library_path("./thirdparty/libtorch/lib")
#pragma cling load("libtorch")

#pragma cling add_include_path("./thirdparty/nest/include")

#include <torch/torch.h>
#include <torch/csrc/autograd/variable.h>
#include <torch/csrc/autograd/function.h>

#include "xtensor/xbuilder.hpp"
#include "xtensor/xarray.hpp"
#include "xtensor/xtensor.hpp"
#include "nest.h"
#include <vector>
#include <array>

In file included from input_line_14:1:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/torch.h:3:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/all.h:8:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/data.h:3:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/data/dataloader.h:3:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/data/dataloader/stateful.h:3:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/data/dataloader/base.h:3:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/data/dataloader_options.h:4:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/types.h:3:
In file included from ./thirdparty/libtorch/include/ATen/ATen.h:9:
In file included from ./thirdparty/libtorch/include/ATen/Context.h:4:
In file included from ./third

In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/data/dataloader_options.h:4:
In file included from ./thirdparty/libtorch/include/torch/csrc/api/include/torch/types.h:3:
In file included from ./thirdparty/libtorch/include/ATen/ATen.h:9:
In file included from ./thirdparty/libtorch/include/ATen/Context.h:4:
In file included from ./thirdparty/libtorch/include/ATen/Tensor.h:3:
In file included from ./thirdparty/libtorch/include/ATen/core/TensorBody.h:7:
In file included from ./thirdparty/libtorch/include/c10/core/Scalar.h:10:
In file included from ./thirdparty/libtorch/include/c10/core/ScalarType.h:8:
In file included from ./thirdparty/libtorch/include/c10/util/typeid.h:33:
./thirdparty/libtorch/include/c10/util/flat_hash_map.h:924:35: error: expected a qualified name after 'typename'
    template<typename L, typename R>
                                  ^
./thirdparty/kdb-cpp/include/internal/../external/k.h:100:11: note: expanded from macro 'R'
#define R r

Interpreter Error: 

In [3]:
#define HOST_ADDR "127.0.0.1"
#define HOST_PORT 5050

IncrementalExecutor::executeFunction: symbol '_ZN3kdb9ConnectorD1Ev' unresolved while linking function '_GLOBAL__sub_I_cling_module_10'!
You are probably missing the definition of kdb::Connector::~Connector()
Maybe you need to load the corresponding shared library?


In [2]:
typedef nest::Nest<torch::Tensor> TensorNest;

In [3]:
torch::Tensor x = torch::ones({2, 2}, torch::requires_grad());
torch::Tensor y = torch::randn({2, 2});

In [2]:
std::vector<size_t> shape = { 3, 2, 4 };
std::vector<size_t> strides = { 8, 4, 1 };

In [4]:
std::vector<size_t> shape = { 3, 2, 4 };
xt::xarray<double, xt::layout_type::row_major> a(shape);

In [7]:
std::array<size_t, 3> shape = { 3, 2, 4 };
xt::xtensor<double, 3> b(shape);

In [ ]:
// typedef int  I;          // Already defined within k.h
// typedef char *S;         // Already defined within k.h
// I khpu(S hostname, I portNumber, S usernamePassword)
khpu ("localhost", 12345, "username:password");